In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os 
import glob
#import cv2
from tqdm import tqdm_notebook
from random import shuffle
import shutil
import pandas as pd

In [ ]:
PATH_IMAGES = "C:\\Users\\rafae\\Desktop\\Coleta\\data"

IR_FOLDER = "ir-subset-eor"
DEPTH_FOLDER ="depth-subset-eor"

TRAIN = 'train'
VAL = 'val'

EOR = 'EOR'
NEOR = 'NEOR'

DASH = "\\"

IR1_PATTERN = '{:s}_{:s}_ir1_{:s}'
IR2_PATTERN = '{:s}_{:s}_ir2_{:s}'
DEP_PATTERN = '{:s}_{:s}_depth_{:s}'
DEPC_PATTERN = '{:s}_{:s}_color_depth_color_{:s}'
PNG = '.png'

EOR_VALUE = 1
NEOR_VALUE = 0

In [ ]:
def load_data(data, pathX, img_rows=128, img_cols=128, channels=3,
              standardize=False,
              contrast=False,
              equalize=False,
              stretch_img=False,
              data_augmentation=True,
              angle_norm_method=1,
              max_angle=90.0):

    for i, d in enumerate(data):
        folder = "%02d/" % d['id'] + "/"
        file = "frame_" + d['frame'] + "_face_depth.png"

        if channels > 1:
            img = cv2.imread(pathX + folder + file, 1)
        else:
            img = cv2.imread(pathX + folder + file, 0)

        if img is None:
            continue

        for (data_augmentation_idx from xrange(1,11):
            if data_augmentation:
                #data_augmentation_idx = d['data_augmentation']
                rdim = random.randrange(1, 15)

                # Corner Top-left
                if data_augmentation_idx == 1:
                    img = img[:img.shape[0]-rdim, :img.shape[1]-rdim]
                # Corner Top-right
                elif data_augmentation_idx == 2:
                    img = img[:img.shape[0]-rdim, rdim:img.shape[1]]
                # Corner Bottom-left
                elif data_augmentation_idx == 3:
                    img = img[rdim:img.shape[0], :img.shape[1]-rdim]
                # Corner bottom-right
                elif data_augmentation_idx == 4:
                    img = img[rdim:img.shape[0], rdim:img.shape[1]]
                # Center
                elif data_augmentation_idx == 5:
                    img = img[int(rdim/2):img.shape[0]-int(rdim/2), int(rdim/2):img.shape[1]-int(rdim/2)]
                # Top
                elif data_augmentation_idx == 6:
                    img = img[int(rdim/2):, :]
                # Bottom
                elif data_augmentation_idx == 7:
                    img = img[:img.shape[0]-int(rdim/2), :]
                # Left
                elif data_augmentation_idx == 8:
                    img = img[:, int(rdim/2):]
                # Right
                elif data_augmentation_idx == 9:
                    img = img[:, :img.shape[1]-int(rdim/2)]
                elif data_augmentation_idx == 10:
                    noise = np.zeros_like(img, 'float') + np.random.normal(0.0, 5, img.shape)
                    img = (img + noise).astype(np.uint8)
                    img[img < 0] = 0
                    img[img > 255] = 255


        if stretch_img:
            img = stretch(img, 190)

        if equalize:
            cv2.equalizeHist(img.astype(np.uint8), img)

        if contrast:
            img = img.astype(np.float32)
            if channels > 1:
                for i_ch in range(channels):
                    img[..., i_ch] = exposure.rescale_intensity(img[...,i_ch].astype('float'), out_range=(0, 1))
            else:
                p2, p98 = np.percentile(img[img < 100], (2, 98))
                img = exposure.rescale_intensity(img.astype('float'), in_range=(p2, p98), out_range=(0,1))

        if standardize:
            img = img.astype(np.float32)
            if channels > 1:
                for i_ch in range(channels):
                    img[...,i_ch] = preprocessing.scale(img[...,i_ch].astype('float'))
            else:
                img = preprocessing.scale(img.astype('float'))

        img = img.astype(np.float32)


        if img.shape[1] != img_cols or img.shape[0] != img_rows:
            img = cv2.resize(img, (img_cols,img_rows))

        # write image
        

    return X, Y